In [ ]:
import os
import glob
import time
import numpy as np
from PIL import Image
from pathlib import Path
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from skimage.color import rgb2lab, lab2rgb
Path.ls = lambda x: list(x.iterdir())

import torch
from torch import nn, optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.utils import make_grid

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

In [ ]:
from fastai.vision.learner import create_body
from torchvision.models.resnet import resnet18
from fastai.vision.models.unet import DynamicUnet

In [ ]:
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split

# Define the path to the training data
color_path = '/kaggle/input/imagenet-object-localization-challenge/ILSVRC/Data/CLS-LOC/train'

# Get all class directories
class_dirs = [d for d in os.listdir(color_path) if os.path.isdir(os.path.join(color_path, d))]

max_images = 10000
all_images = []
np.random.seed(100)

# Gather up to 1000 images total
for class_dir in class_dirs:
    if len(all_images) >= max_images:
        break
    class_path_pattern = os.path.join(color_path, class_dir, '*.JPEG')
    class_images = glob.glob(class_path_pattern)
    np.random.shuffle(class_images)  # Shuffle to get a random sample from this class
    needed = max_images - len(all_images)
    to_add = class_images[:needed]
    all_images.extend(to_add)

print("Total images collected:", len(all_images))

# Split into training and validation sets (80% train, 20% val)
train_paths, val_paths = train_test_split(all_images, test_size=0.2, random_state=123)

print("Total training images:", len(train_paths))
print("Total validation images:", len(val_paths))

# Display a few training images
_, axes = plt.subplots(4, 4, figsize=(12, 12))
for ax, img_path in zip(axes.flatten(), train_paths[:16]):
    img = Image.open(img_path)
    ax.imshow(img)
    ax.axis("off")
plt.tight_layout()
plt.show()

In [ ]:
SIZE = 256

class ColorizationDataset(Dataset):
    def __init__(self, paths, split='train'):
        if split == 'train':
            self.transforms = transforms.Compose([
                transforms.Resize((SIZE, SIZE), Image.BICUBIC),
                transforms.RandomHorizontalFlip(),
            ])
        else:
            self.transforms = transforms.Resize((SIZE, SIZE), Image.BICUBIC)
        
        self.split = split
        self.size = SIZE
        self.paths = paths
    
    def __getitem__(self, idx):
        img = Image.open(self.paths[idx]).convert("RGB")
        img = self.transforms(img)
        img = np.array(img)
        img_lab = rgb2lab(img).astype("float32")
        img_lab = transforms.ToTensor()(img_lab)
        L = img_lab[[0], ...] / 50. - 1.
        ab = img_lab[[1, 2], ...] / 110.
        return {'L': L, 'ab': ab}
    
    def __len__(self):
        return len(self.paths)

def make_dataloaders(paths, split='train', batch_size=16, n_workers=4, pin_memory=True):
    dataset = ColorizationDataset(paths, split=split)
    dataloader = DataLoader(dataset, batch_size=batch_size, 
                            num_workers=n_workers, pin_memory=pin_memory, shuffle=(split=='train'))
    return dataloader

train_dl = make_dataloaders(train_paths, split='train', batch_size=16)
val_dl = make_dataloaders(val_paths, split='val', batch_size=16)

data = next(iter(train_dl))
Ls, abs_ = data['L'], data['ab']
print("Train batch shapes:", Ls.shape, abs_.shape)
print("Number of batches:", len(train_dl), "train,", len(val_dl), "val")

In [ ]:
class UnetBlock(nn.Module):
    def __init__(self, nf, ni, submodule=None, input_c=None, dropout=False,
                 innermost=False, outermost=False):
        super().__init__()
        self.outermost = outermost
        if input_c is None: input_c = nf
        downconv = nn.Conv2d(input_c, ni, kernel_size=4,
                             stride=2, padding=1, bias=False)
        downrelu = nn.LeakyReLU(0.2, True)
        downnorm = nn.BatchNorm2d(ni)
        uprelu = nn.ReLU(True)
        upnorm = nn.BatchNorm2d(nf)
        
        if outermost:
            upconv = nn.ConvTranspose2d(ni * 2, nf, kernel_size=4,
                                        stride=2, padding=1)
            down = [downconv]
            up = [uprelu, upconv, nn.Tanh()]
            model = down + [submodule] + up
        elif innermost:
            upconv = nn.ConvTranspose2d(ni, nf, kernel_size=4,
                                        stride=2, padding=1, bias=False)
            down = [downrelu, downconv]
            up = [uprelu, upconv, upnorm]
            model = down + up
        else:
            upconv = nn.ConvTranspose2d(ni * 2, nf, kernel_size=4,
                                        stride=2, padding=1, bias=False)
            down = [downrelu, downconv, downnorm]
            up = [uprelu, upconv, upnorm]
            if dropout: up += [nn.Dropout(0.5)]
            model = down + [submodule] + up
        self.model = nn.Sequential(*model)
    
    def forward(self, x):
        if self.outermost:x
            return self.model(x)
        else:
            return torch.cat([x, self.model(x)], 1)

class Unet(nn.Module):
    def __init__(self, input_c=1, output_c=2, n_down=8, num_filters=64):
        super().__init__()
        unet_block = UnetBlock(num_filters * 8, num_filters * 8, innermost=True)
        for _ in range(n_down - 5):
            unet_block = UnetBlock(num_filters * 8, num_filters * 8, submodule=unet_block, dropout=True)
        out_filters = num_filters * 8
        for _ in range(3):
            unet_block = UnetBlock(out_filters // 2, out_filters, submodule=unet_block)
            out_filters //= 2
        self.model = UnetBlock(output_c, out_filters, input_c=input_c, submodule=unet_block, outermost=True)
    
    def forward(self, x):
        return self.model(x)

class PatchDiscriminator(nn.Module):
    def __init__(self, input_c, num_filters=64, n_down=3):
        super().__init__()
        model = [self.get_layers(input_c, num_filters, norm=False)]
        model += [self.get_layers(num_filters * 2 ** i, num_filters * 2 ** (i + 1), s=1 if i == (n_down-1) else 2) 
                  for i in range(n_down)]
        model += [self.get_layers(num_filters * 2 ** n_down, 1, s=1, norm=False, act=False)]
        self.model = nn.Sequential(*model)
        
    def get_layers(self, ni, nf, k=4, s=2, p=1, norm=True, act=True):
        layers = [nn.Conv2d(ni, nf, k, s, p, bias=not norm)]
        if norm: layers += [nn.BatchNorm2d(nf)]
        if act: layers += [nn.LeakyReLU(0.2, True)]
        return nn.Sequential(*layers)
    
    def forward(self, x):
        return self.model(x)

In [ ]:
class MultiscaleUnet(nn.Module):
    def __init__(self, input_c=3, output_c=2, n_down=8, num_filters=64):
        super().__init__()
        self.unet = Unet(input_c=input_c, output_c=output_c, n_down=n_down, num_filters=num_filters)

    def forward(self, L):
        # Generate multiscale inputs
        L_low = nn.functional.interpolate(L, scale_factor=0.5, mode='bilinear', align_corners=False)
        L_high = nn.functional.interpolate(L, scale_factor=2.0, mode='bilinear', align_corners=False)
        conditional_input = torch.cat((L, L_low, L_high), dim=1)  # Concatenate original, low, and high-res inputs
        return self.unet(conditional_input)

class SelfAttention(nn.Module):
    def __init__(self, in_dim):
        super(SelfAttention, self).__init__()
        self.query = nn.Conv2d(in_dim, in_dim // 8, kernel_size=1)
        self.key = nn.Conv2d(in_dim, in_dim // 8, kernel_size=1)
        self.value = nn.Conv2d(in_dim, in_dim, kernel_size=1)
        self.gamma = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        batch, channels, height, width = x.size()
        query = self.query(x).view(batch, -1, width * height).permute(0, 2, 1)
        key = self.key(x).view(batch, -1, width * height)
        attention = torch.bmm(query, key)
        attention = nn.functional.softmax(attention, dim=-1)
        value = self.value(x).view(batch, -1, width * height)
        out = torch.bmm(value, attention.permute(0, 2, 1)).view(batch, channels, height, width)
        out = self.gamma * out + x
        return out

class ConditionalUnetWithFeatures(nn.Module):
    def __init__(self, input_c=1, output_c=2, n_down=8, num_filters=64, feature_dim=512):
        super().__init__()
        # ResNet for feature extraction
        self.feature_extractor = torchvision.models.resnet18(pretrained=True)
        self.feature_extractor.fc = nn.Identity()  # Remove classification head
        self.feature_extractor.eval()
        for param in self.feature_extractor.parameters():
            param.requires_grad = False

        # Generator with multiscale inputs
        self.unet = MultiscaleUnet(input_c=input_c + 3, output_c=output_c, n_down=n_down, num_filters=num_filters)

    def forward(self, L):
        # Extract semantic features
        with torch.no_grad():
            features = self.feature_extractor(L.repeat(1, 3, 1, 1))  # Pass grayscale image through ResNet
        features = features.unsqueeze(-1).unsqueeze(-1).repeat(1, 1, SIZE, SIZE)  # Reshape to spatial dimensions
        conditional_input = torch.cat((L, features), dim=1)  # Concatenate grayscale and features
        return self.unet(conditional_input)

class ConditionalDiscriminatorWithFeatures(PatchDiscriminator):
    def __init__(self, input_c=3, feature_dim=512, num_filters=64, n_down=3):
        super().__init__(input_c + feature_dim // 64, num_filters, n_down)
        self.feature_extractor = torchvision.models.resnet18(pretrained=True)
        self.feature_extractor.fc = nn.Identity()
        self.feature_extractor.eval()
        for param in self.feature_extractor.parameters():
            param.requires_grad = False

    def forward(self, x, L):
        # Extract features
        with torch.no_grad():
            features = self.feature_extractor(L.repeat(1, 3, 1, 1))
        features = features.unsqueeze(-1).unsqueeze(-1).repeat(1, 1, SIZE, SIZE)
        conditional_input = torch.cat((x, features), dim=1)  # Concatenate image and features
        return super().forward(conditional_input)

In [ ]:
class GANLoss(nn.Module):
    def __init__(self, gan_mode='vanilla', real_label=1.0, fake_label=0.0):
        super().__init__()
        self.register_buffer('real_label', torch.tensor(real_label))
        self.register_buffer('fake_label', torch.tensor(fake_label))
        if gan_mode == 'vanilla':
            self.loss = nn.BCEWithLogitsLoss()
        elif gan_mode == 'lsgan':
            self.loss = nn.MSELoss()
    
    def get_labels(self, preds, target_is_real):
        if target_is_real:
            labels = self.real_label
        else:
            labels = self.fake_label
        return labels.expand_as(preds)
    
    def forward(self, preds, target_is_real):
        labels = self.get_labels(preds, target_is_real)
        loss = self.loss(preds, labels)
        return loss

def init_weights(net, init='norm', gain=0.02):
    def init_func(m):
        classname = m.__class__.__name__
        if hasattr(m, 'weight') and 'Conv' in classname:
            if init == 'norm':
                nn.init.normal_(m.weight.data, mean=0.0, std=gain)
            elif init == 'xavier':
                nn.init.xavier_normal_(m.weight.data, gain=gain)
            elif init == 'kaiming':
                nn.init.kaiming_normal_(m.weight.data, a=0, mode='fan_in')
            
            if hasattr(m, 'bias') and m.bias is not None:
                nn.init.constant_(m.bias.data, 0.0)
        elif 'BatchNorm2d' in classname:
            nn.init.normal_(m.weight.data, 1., gain)
            nn.init.constant_(m.bias.data, 0.)
            
    net.apply(init_func)
    print(f"model initialized with {init} initialization")
    return net

def init_model(model, device):
    model = model.to(device)
    model = init_weights(model)
    return model

class AverageMeter:
    def __init__(self):
        self.reset()
        
    def reset(self):
        self.count, self.avg, self.sum = [0.] * 3
    
    def update(self, val, count=1):
        self.count += count
        self.sum += count * val
        self.avg = self.sum / self.count

def create_loss_meters():
    loss_D_fake = AverageMeter()
    loss_D_real = AverageMeter()
    loss_D = AverageMeter()
    loss_G_GAN = AverageMeter()
    loss_G_L1 = AverageMeter()
    loss_G = AverageMeter()
    
    return {'loss_D_fake': loss_D_fake,
            'loss_D_real': loss_D_real,
            'loss_D': loss_D,
            'loss_G_GAN': loss_G_GAN,
            'loss_G_L1': loss_G_L1,
            'loss_G': loss_G}

def update_losses(model, loss_meter_dict, count):
    for loss_name, loss_meter in loss_meter_dict.items():
        loss = getattr(model, loss_name)
        loss_meter.update(loss.item(), count=count)

In [ ]:
def lab_to_rgb(L, ab):
    L = (L + 1.) * 50.
    ab = ab * 110.
    Lab = torch.cat([L, ab], dim=1).permute(0, 2, 3, 1).cpu().numpy()
    rgb_imgs = []
    for img in Lab:
        img_rgb = lab2rgb(img)
        rgb_imgs.append(img_rgb)
    return np.stack(rgb_imgs, axis=0)
    
def visualize(model, data, save=True):
    model.net_G.eval()
    with torch.no_grad():
        model.setup_input(data)
        model.forward()
    model.net_G.train()
    fake_color = model.fake_color.detach()
    real_color = model.ab
    L = model.L
    fake_imgs = lab_to_rgb(L, fake_color)
    real_imgs = lab_to_rgb(L, real_color)
    fig = plt.figure(figsize=(15, 12))  # Adjusted height for row titles
    rows, cols = 3, 5  # 3 rows (Grayscale, Model generated, Actual), up to 5 columns

    # Add row titles
    plt.subplot(rows, cols, 1).set_title("Grayscale Image", fontsize=16, loc='left')
    plt.subplot(rows, cols, cols + 1).set_title("Model Generated Image", fontsize=16, loc='left')
    plt.subplot(rows, cols, 2 * cols + 1).set_title("Actual Image", fontsize=16, loc='left')

    for i in range(min(5, L.size(0))):
        # Grayscale Image (Row 1)
        ax = plt.subplot(rows, cols, i + 1)
        ax.imshow(L[i][0].cpu(), cmap='gray')
        ax.axis("off")

        # Model Generated Image (Row 2)
        ax = plt.subplot(rows, cols, i + 1 + cols)
        ax.imshow(fake_imgs[i])
        ax.axis("off")

        # Actual Image (Row 3)
        ax = plt.subplot(rows, cols, i + 1 + 2 * cols)
        ax.imshow(real_imgs[i])
        ax.axis("off")

    plt.tight_layout()
    plt.show()
    if save:
        fig.savefig(f"images_new/colorization_{time.time()}.png")

def log_results(loss_meter_dict, log_file, epoch, iteration):
    """
    Log the training results to a CSV file and print them to the console.
    """
    with open(log_file, mode='a', newline='') as file:
        writer = csv.writer(file)
        for loss_name, loss_meter in loss_meter_dict.items():
            print(f"{loss_name}: {loss_meter.avg:.5f}")  # Print to console
            # Append the results to the CSV file
            writer.writerow([epoch, iteration, loss_name, loss_meter.avg])

In [ ]:
class ConditionalGANModel(nn.Module):
    def __init__(self, net_G=None, lr_G=2e-4, lr_D=2e-4, beta1=0.5, beta2=0.999, lambda_L1=100.):
        super().__init__()
        
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.lambda_L1 = lambda_L1
        
        # Initialize or use the provided generator
        if net_G is None:
            self.net_G = init_model(Unet(input_c=1, output_c=2, n_down=8, num_filters=64), self.device)
        else:
            self.net_G = net_G.to(self.device)
        
        # Initialize the discriminator with conditioning
        self.net_D = init_model(PatchDiscriminator(input_c=3, n_down=3, num_filters=64), self.device)
        
        # Define losses
        self.GANcriterion = GANLoss(gan_mode='vanilla').to(self.device)
        self.L1criterion = nn.L1Loss()
        
        # Optimizers for generator and discriminator
        self.opt_G = optim.Adam(self.net_G.parameters(), lr=lr_G, betas=(beta1, beta2))
        self.opt_D = optim.Adam(self.net_D.parameters(), lr=lr_D, betas=(beta1, beta2))
    
    def set_requires_grad(self, model, requires_grad=True):
        for p in model.parameters():
            p.requires_grad = requires_grad
        
    def setup_input(self, data):
        self.L = data['L'].to(self.device)
        self.ab = data['ab'].to(self.device)
        
    def forward(self):
        self.fake_color = self.net_G(self.L)
    
    def backward_D(self):
        # Concatenate grayscale input (L) with fake/generated colors
        fake_image = torch.cat([self.L, self.fake_color], dim=1)
        fake_preds = self.net_D(fake_image.detach())
        self.loss_D_fake = self.GANcriterion(fake_preds, False)
        
        # Concatenate grayscale input (L) with real colors
        real_image = torch.cat([self.L, self.ab], dim=1)
        real_preds = self.net_D(real_image)
        self.loss_D_real = self.GANcriterion(real_preds, True)
        
        # Total discriminator loss
        self.loss_D = (self.loss_D_fake + self.loss_D_real) * 0.5
        self.loss_D.backward()
    
    def backward_G(self):
        fake_image = torch.cat([self.L, self.fake_color], dim=1)
        fake_preds = self.net_D(fake_image)
        self.loss_G_GAN = self.GANcriterion(fake_preds, True)
        
        # L1 loss between the generated and actual colors
        self.loss_G_L1 = self.L1criterion(self.fake_color, self.ab) * self.lambda_L1
        
        # Total generator loss
        self.loss_G = self.loss_G_GAN + self.loss_G_L1
        self.loss_G.backward()
    
    def optimize(self):
        # Update discriminator
        self.forward()
        self.net_D.train()
        self.set_requires_grad(self.net_D, True)
        self.opt_D.zero_grad()
        self.backward_D()
        self.opt_D.step()
        
        # Update generator
        self.net_G.train()
        self.set_requires_grad(self.net_D, False)
        self.opt_G.zero_grad()
        self.backward_G()
        self.opt_G.step()

In [ ]:
import random

# Ensure DataLoader tensors are on the correct device
def move_batch_to_device(batch, device):
    return {key: value.to(device) for key, value in batch.items()}

import random
def train_model(model, train_dl, epochs, display_every=200, log_file="logs_training_final.csv"):
    # Prepare the CSV file for logging
    with open(log_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        # Write the header
        writer.writerow(["Epoch", "Iteration", "Loss Name", "Average Loss"])

    for e in range(epochs):
        loss_meter_dict = create_loss_meters()
        i = 0
        # Generate a random interval for image visualization for this epoch
        visualize_every = random.randint(1, 499)
        for batch_data in tqdm(train_dl):
            model.setup_input(batch_data)
            model.optimize()
            update_losses(model, loss_meter_dict, count=batch_data['L'].size(0))
            i += 1

            # Log results at fixed intervals
            if i % display_every == 0:
                print(f"\nEpoch {e+1}/{epochs}")
                print(f"Iteration {i}/{len(train_dl)}")
                log_results(loss_meter_dict, log_file, epoch=e + 1, iteration=i)

            # Visualize images at random intervals
            if i % visualize_every == 0:
                print(f"Visualizing at random iteration {i} (interval: {visualize_every})")
                # Fetch a new batch of validation data for visualization
                data = next(iter(val_dl))
                visualize(model, data, save=True)

In [ ]:
def build_res_unet(n_input=1, n_output=2, size=256):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # First, instantiate the resnet18 model with pretrained weights
    backbone = resnet18(pretrained=True)
    # Now pass this instantiated model to create_body
    body = create_body(backbone, n_in=n_input, cut=-2)
    net_G = DynamicUnet(body, n_output, (size, size)).to(device)
    return net_G


def pretrain_generator(net_G, train_dl, opt, criterion, epochs):
    for e in range(epochs):
        loss_meter = AverageMeter()
        for batch_data in tqdm(train_dl):
            L, ab = batch_data['L'].to(device), batch_data['ab'].to(device)
            preds = net_G(L)
            loss = criterion(preds, ab)
            opt.zero_grad()
            loss.backward()
            opt.step()
            
            loss_meter.update(loss.item(), L.size(0))
            
        print(f"Epoch {e + 1}/{epochs}")
        print(f"L1 Loss: {loss_meter.avg:.5f}")

In [ ]:
# Pretrain the generator
net_G = build_res_unet(n_input=1, n_output=2, size=256)
opt_g = optim.Adam(net_G.parameters(), lr=1e-4)
criterion_L1 = nn.L1Loss()

print("Pretraining the generator with L1 loss...")
pretrain_generator(net_G, train_dl, opt_g, criterion_L1, epochs=20)

# Save the pretrained generator weights
torch.save(net_G.state_dict(), "res18-unet-v2.pt")
print("Pretraining complete and weights saved.")

In [ ]:
"""
# Define the path to the pretrained weights
pretrained_weights_path = "/kaggle/working/res18-unet.pt"  # Update this path if the weights are saved elsewhere

# Load the pretrained generator
print("Loading pretrained generator weights...")
net_G = build_res_unet(n_input=1, n_output=2, size=256)  # Build the generator architecture
net_G.load_state_dict(torch.load(pretrained_weights_path, map_location=device))  # Load weights
net_G = net_G.to(device)  # Move the model to the appropriate device (GPU/CPU)
net_G.eval()  # Set the model to evaluation mode (optional, for inference)

print("Pretrained generator loaded successfully!")
"""

In [ ]:
import os
import csv
# Ensure the images directory exists
#os.makedirs("/kaggle/working/images", exist_ok=True)
#!rm /kaggle/working/images/colorization_1733694174.787311.png
!mkdir images_new

In [ ]:
"""
# Load the pretrained generator from the working directory
net_G = build_res_unet(n_input=1, n_output=2, size=256)

# Ensure the model is on the GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net_G = net_G.to(device)

# Load the pretrained weights into the generator
pretrained_weights_path = "/kaggle/input/res18-v2/other/default/1/res18-unet-v2.pt"  # Path to the weights file
net_G.load_state_dict(torch.load(pretrained_weights_path, map_location=device))
print("Pretrained weights loaded into the generator.")

# Create the main model with the pretrained net_G
model = MainModel(net_G=net_G, lr_G=2e-4, lr_D=2e-4, beta1=0.5, beta2=0.999, lambda_L1=100.)
model = model.to(device)  # Ensure the full model is on the GPU
print("Main model created and moved to the GPU (if available).")

# Train the model
print("Training the full model (GAN + L1) now...")
train_model(model, train_dl, epochs=50, display_every=200)
print("Training complete.")
"""

In [ ]:
# Load the pretrained generator
net_G = build_res_unet(n_input=1, n_output=2, size=256)
net_G.load_state_dict(torch.load("/kaggle/working/res18-unet-v2.pt", map_location=device))

# Create the main model with the pretrained net_G
model = ConditionalGANModel(net_G=net_G, lr_G=2e-4, lr_D=2e-4, beta1=0.5, beta2=0.999, lambda_L1=100.)
print("Training the full model (GAN + L1) now...")
train_model(model, train_dl, epochs=50, display_every=200)
print("Training complete.")

In [ ]:
# Save the generator weights
generator_save_path = "/kaggle/working/net_G_final.pt"
torch.save(model.net_G.state_dict(), generator_save_path)
print(f"Generator weights saved to {generator_save_path}")

# Save the entire model
main_model_save_path = "/kaggle/working/main_model_final.pt"
torch.save({
    'net_G': model.net_G.state_dict(),
    'net_D': model.net_D.state_dict(),
    'opt_G': model.opt_G.state_dict(),
    'opt_D': model.opt_D.state_dict()
}, main_model_save_path)
print(f"Full model saved to {main_model_save_path}")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from skimage.color import lab2rgb

def lab_to_rgb(L, ab):
    """
    Convert L and ab channels to RGB images.
    Args:
        L: Lightness channel (normalized to [-1, 1])
        ab: Color channels (normalized to [-1, 1])
    Returns:
        RGB image
    """
    L = (L + 1.) * 50.  # Rescale L to [0, 100]
    ab = ab * 110.      # Rescale ab to [-110, 110]
    Lab = np.concatenate([L, ab], axis=0).transpose(1, 2, 0)  # Combine channels
    return lab2rgb(Lab)

def plot_results(original_images, grayscale_images, generated_images, num_images=10):
    """
    Display original, grayscale, and generated images side by side.
    Args:
        original_images: List or array of original color images.
        grayscale_images: List or array of grayscale images.
        generated_images: List or array of generated colorized images.
        num_images: Number of images to display.
    """
    plt.figure(figsize=(15, 5 * num_images))
    for i in range(num_images):
        original = original_images[i]
        grayscale = grayscale_images[i]
        generated = generated_images[i]

        # Plot original image
        plt.subplot(num_images, 3, i * 3 + 1)
        plt.imshow(original)
        plt.title("Original Image")
        plt.axis("off")

        # Plot grayscale image
        plt.subplot(num_images, 3, i * 3 + 2)
        plt.imshow(grayscale[0], cmap="gray")
        plt.title("Grayscale Image")
        plt.axis("off")

        # Plot generated image
        plt.subplot(num_images, 3, i * 3 + 3)
        plt.imshow(generated)
        plt.title("Generated Image")
        plt.axis("off")
    plt.tight_layout()
    plt.show()

# Example usage
# Assume `original_images` are in RGB, `grayscale_images` are single-channel, and `predicted_images` are the model outputs.
# Convert predictions from LAB to RGB before displaying.

# Placeholder for 10 original images, grayscale inputs, and generated predictions
original_images = [lab_to_rgb(data['L'][i].cpu().numpy(), data['ab'][i].cpu().numpy()) for i in range(10)]
grayscale_images = [data['L'][i].cpu().numpy() for i in range(10)]
generated_images = [lab_to_rgb(data['L'][i].cpu().numpy(), model.fake_color[i].detach().cpu().numpy()) for i in range(10)]

# Display results
plot_results(original_images, grayscale_images, generated_images, num_images=10)

In [ ]:
from sklearn.model_selection import KFold
from torch.utils.data import Subset, DataLoader
import copy

def cross_validate(model_class, dataset, k_folds=5, epochs=10, batch_size=16, lr=1e-4, lambda_L1=100.):
    """
    Perform K-fold cross-validation for the given model and dataset.

    Args:
        model_class: The model class to be instantiated for each fold.
        dataset: The dataset to split into K folds.
        k_folds: Number of folds for cross-validation.
        epochs: Number of epochs to train each fold.
        batch_size: Batch size for DataLoader.
        lr: Learning rate for optimizer.
        lambda_L1: Weight for L1 loss in generator.

    Returns:
        Average loss and metrics over all folds.
    """
    kfold = KFold(n_splits=k_folds, shuffle=True, random_state=123)
    fold_results = []

    # Split dataset into K folds
    for fold, (train_idx, val_idx) in enumerate(kfold.split(dataset)):
        print(f"\nFold {fold + 1}/{k_folds}")

        # Create dataloaders for this fold
        train_subset = Subset(dataset, train_idx)
        val_subset = Subset(dataset, val_idx)
        train_dl = DataLoader(train_subset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
        val_dl = DataLoader(val_subset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

        # Initialize model
        model = model_class(lr_G=lr, lambda_L1=lambda_L1)
        model = model.to(device)

        # Train the model
        for epoch in range(epochs):
            model.train()
            for batch_data in train_dl:
                model.setup_input(batch_data)
                model.optimize()

            # Validate the model
            model.eval()
            val_loss_meter = AverageMeter()
            with torch.no_grad():
                for batch_data in val_dl:
                    model.setup_input(batch_data)
                    model.forward()
                    val_loss = model.L1criterion(model.fake_color, model.ab)
                    val_loss_meter.update(val_loss.item(), batch_data['L'].size(0))

            print(f"Fold {fold + 1}, Epoch {epoch + 1}/{epochs}, Validation L1 Loss: {val_loss_meter.avg:.5f}")

        # Store results for this fold
        fold_results.append(val_loss_meter.avg)

    # Compute average loss across all folds
    avg_loss = np.mean(fold_results)
    print(f"\nCross-validation complete. Average Validation L1 Loss: {avg_loss:.5f}")
    return avg_loss

In [ ]:
# Instantiate the dataset
dataset = ColorizationDataset(paths=train_paths, split='train')

# Perform cross-validation
avg_loss = cross_validate(
    model_class=MainModel,
    dataset=dataset,
    k_folds=5,         # 5-fold cross-validation
    epochs=10,         # 10 epochs per fold
    batch_size=16,     # Batch size
    lr=1e-4,           # Learning rate
    lambda_L1=100.     # L1 loss weight
)

print(f"Final Cross-Validation Average Loss: {avg_loss:.5f}")